<a href="https://colab.research.google.com/github/Hritik1100/Lunar-Lander-using-AI/blob/main/Lunar_Landing_Deep_Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.5 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446659 sha256=1017a409dfa32b4c7d73d6b87f189d85f9f82703eb8159b27e59bde8f4d40cbb
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installe

### Importing the libraries

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [ ]:
class Network(nn.Module):

  def __init__(self, state_size, action_size, seed = 42):    ## 1st argument is Input Vector (Observation State) and 2nd argument is output vector (Action State)
      super(Network, self).__init__()

      self.seed = torch.manual_seed(seed)
      self.fc1 = nn.Linear(state_size, 64)  ## Input to the Fully Connected Layer is the observation state and the output is 64 after doing trail and error
      self.fc2 = nn.Linear(64, 64)          ## Created another FC layer with input as the output of the previous FC layer
      self.fc3 = nn.Linear(64, action_size)


  def forward(self, state):

    ## 1st 2 lines: Propagate the signal from the input layer to the 1st fully connected layer with the relu activation function
    x = self.fc1(state)
    x = F.relu(x)

    ## Next 2 lines: Does the same thing as above for the 2nd FC layer
    x = self.fc2(x)
    x = F.relu(x)

    ## Take the output of 2nd FC layer and provide as input to the 3rd FC layer
    return self.fc3(x)


## Part 2 - Training the AI

### Setting up the environment

In [ ]:
## Calculate observation space and action space of the environment

import gymnasium as gym
env = gym.make("LunarLander-v2")
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]

number_actions = env.action_space.n

print("State Shape: ", state_shape)
print("State Size: ", state_size)
print("Number of actions: ", number_actions)

State Shape:  (8,)
State Size:  8
Number of actions:  4


### Initializing the hyperparameters

In [ ]:
learning_rate = 5e-4              ##(5 * (10^-4))
minibatch_size = 100              ## Batch size to update the model parameters
discount_factor =  0.99           ## Gamma

replay_buffer_size = int(1e5)     ## AI memory size
interpolation_parameter = 1e-3    ## For soft update

### Implementing Experience Replay

In [ ]:
class ReplayMemory(object):

  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []   ## List which stores experiences each one containing action, state, reward, next state, and dones


  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]    ## Delete the oldest memory which will be at the 0th index

  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)

    return states, next_states, actions, rewards, dones

### Implementing the DQN class

In [ ]:
class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = Network(state_size, action_size).to(self.device)
    self.target_qnetwork = Network(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4
    if self.t_step==0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)



## Why we do soft update is to not abruptly change the target network parameters
  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)     ## Calculate weighted average of local and target parameters
                                                                                                                                    ## to update target network parameters


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Initializing the DQN agent

In [ ]:
agent = Agent(state_size, number_actions)

### Training the DQN agent

In [ ]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value = 0.995    ## This value should be close to 1
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(number_episodes):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)      ## Select action by making use of epsilon greedy approach
    next_state, reward, done, _, _ = env.step(action)  ## Apply selected action to the current state to get next state and its associated rewards
    agent.step(state, action, reward, next_state, done)    ## To make the agent learn through back propagation
    state = next_state
    score += reward                                       ## Reward is added which makes up to the total score
    if done:
      break

  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)

  print("\rEpisode {}\t Average Score: {:.2f}".format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print("\rEpisode {}\t Average Score: {:.2f}".format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print("\nEnvironment solved in {:d} episode!\tAverage Score: {:.2f}".format(episode - 100, np.mean(scores_on_100_episodes)), end = "")
    torch.save(agent.local_qnetwork.state_dict(), "checkpoint.pth")
    break

Episode 0	 Average Score: -398.56
Episode 100	 Average Score: -183.50
Episode 200	 Average Score: -122.58
Episode 300	 Average Score: -25.68
Episode 400	 Average Score: -10.83
Episode 500	 Average Score: 141.15
Episode 591	 Average Score: 200.23
 Environment solved in 491 episode!	Average Score: 200.23

## Part 3 - Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
